# 0. Import required packages.

In [1]:
import copy
import time
import numpy as np

# 1. Define a `Game` Class for containing the Tic-Tac-Toe game.

In [2]:
class Game(object):
    """A tic-tac-toe game."""

    def __init__(self, grid):
        """Instances differ by their grid marks."""
        self.grid = copy.deepcopy(grid) # No aliasing!

    def display(self):
        """Print the game board."""
        print("    0    1    2  ")
        for i, row in enumerate(self.grid):
            print(i, row)

        print("\n")
        
    def moves(self):
        """Return a list of possible moves given the current marks.
        
        Parameters
        ----------
            None
        
        Returns
        -------
            possible_moves: List of Tuples containing (row, column) of available spaces on the grid.
        
        Author: Miguel Agueda-Cabral
        """
        
        possible_moves = []
        for row_index, row in enumerate(self.grid):
            for column_index, value in enumerate(row):
                if value == '-':
                    position = (row_index, column_index)
                    possible_moves.append(position)
        
        return possible_moves

    def neighbor(self, move, mark):
        """Return a Game instance like this one but with one move made.
        
        Parameters
        ----------
            move: Tuple containing (row, column) indices of move to be made.
            mark: String identity of player ('X', 'O'). 
        
        Returns
        -------
            neighbor_game: A copy of `self` with `move` applied.
        
        Author: Miguel Agueda-Cabral
        """
        
        neighbor_game = Game(self.grid)
        (row_idx, col_idx) = move
        neighbor_game.grid[row_idx][col_idx] = mark
        
        return copy.deepcopy(neighbor_game)

    def utility(self): # author Walter Jordan
        """Return the minimax utility value of this game:
        1 = X win, -1 = O win, 0 = tie, None = not over yet."""
        # need to check for positions, I should just define it statically even if that is inefficient
        winpos = [[(0,0),(0,1),(0,2)], [(1,0),(1,1),(1,2)], [(2,0),(2,1),(2,2)], 
                  [(0,0),(1,0),(2,0)], [(0,1),(1,1),(2,1)], [(0,2),(1,2),(2,2)], 
                  [(0,0),(1,1),(2,2)], [(0,2),(1,1),(2,0)]]
#         blank_found = False
        for x in winpos:
            x1 = x[0][0]
            y1 = x[0][1]
            x2 = x[1][0]
            y2 = x[1][1]
            x3 = x[2][0]
            y3 = x[2][1]
            if self.grid[x1][y1] == self.grid[x2][y2] == self.grid[x3][y3]:
                if self.grid[x1][y1] == 'X':
                    return 1
                if self.grid[x1][y1] == 'O':
                    return -1
#                 if self.grid[x1][y1] == '-':
#                     blank_found = True
                    

#         if '-' not in self.grid:
#         if not blank_found:
        blank_found = False
        for row in self.grid:
            if '-' in row:
                blank_found = True
                
        if not blank_found:
            return 0
        return None

   # 2. Define an `Agent` Class which can manipulate and solve an instance of `Game`.

In [3]:
class Agent(object):
    """Knows how to play tic-tac-toe."""

    def __init__(self, mark):
        """Agents use either X or O."""
        self.mark = mark
        
    def maxvalue(self, game, opponent):
        """ Compute the highest utility this game can have.

        Author: Miguel Agueda-Cabral.
        """
        possibles = game.moves()  # Get a list of all possible moves.
        bestlocation = None
        bestval = -np.inf  # Initialize bestval to negative infinity.
        
        for x in possibles:
            temp_game = game.neighbor(x, 'X')  # Compute maximal move for 'X' player.
            util = temp_game.utility()
            
            if util is not None:
                val = util
            else:
                val, _ = self.minvalue(temp_game, opponent)
            if val > bestval:
                bestval = val
                bestlocation = x
        return bestval, bestlocation


    def minvalue(self, game, opponent): # author Walter Jordan
        """Compute the lowest utility this game can have."""
        possibles = game.moves()
        bestlocation = None
        bestval = None
        
        for x in possibles:
            temp_game = game.neighbor(x, 'O')
            util = temp_game.utility()
            val = None
            
            if util is not None:
                val = util
            else:
                val, _ = self.maxvalue(temp_game, opponent)
            if bestval == None or val < bestval:
                bestval = val
                bestlocation = x
        return bestval, bestlocation
            



In [4]:
def get_human_input(game):
    """ Get human input for Tic-Tac-Toe.
    
    Parameters
    ----------
        game: Current game for human player to play on.
    
    Returns
    -------
        human_move: Legal, Human-chosen move to be played.
    
    Author: Miguel Agueda-Cabral
    """
    
    move_not_legal = True
    human_move = [-1, -1]
    legal_moves = game.moves()
            
    while(move_not_legal):
        human_move[0] = int(input("row: (0,1,2): ").strip(' '))
        human_move[1] = int(input("col: (0,1,2): ").strip(' '))
                
        if tuple(human_move) in legal_moves:
            move_not_legal = False
        else:
            print(F"The move, {human_move}, is not legal!\n")
    
    return tuple(human_move)

In [5]:
def main():
    """Create a game and have two agents play it."""
    
    human_player = input("Human player? (y/n): ").lower()

    game = Game([['-','-','-'], ['-','-','-'], ['-','-','-']])
    game.display()
    

    maxplayer = Agent('X')
    minplayer = Agent('O')

    while True:
        (value, move) = maxplayer.maxvalue(game, minplayer)
        game = game.neighbor(move, maxplayer.mark)
        time.sleep(1)
        game.display()
        
        if game.utility() is not None:
            print("BREAK")
            break
        
        if human_player == 'y':
            move = get_human_input(game)
            
        else:    
            (value, move) = minplayer.minvalue(game, maxplayer)
            
        game = game.neighbor(move, minplayer.mark)
        time.sleep(1)
        game.display()

        if game.utility() is not None:
            print("BREAK")
            break

if __name__ == '__main__':
    main()

Human player? (y/n): y
    0    1    2  
0 ['-', '-', '-']
1 ['-', '-', '-']
2 ['-', '-', '-']


    0    1    2  
0 ['X', '-', '-']
1 ['-', '-', '-']
2 ['-', '-', '-']


row: (0,1,2): 1
col: (0,1,2): 1
    0    1    2  
0 ['X', '-', '-']
1 ['-', 'O', '-']
2 ['-', '-', '-']


    0    1    2  
0 ['X', 'X', '-']
1 ['-', 'O', '-']
2 ['-', '-', '-']


row: (0,1,2): 0
col: (0,1,2): 2
    0    1    2  
0 ['X', 'X', 'O']
1 ['-', 'O', '-']
2 ['-', '-', '-']


    0    1    2  
0 ['X', 'X', 'O']
1 ['-', 'O', '-']
2 ['X', '-', '-']


row: (0,1,2): 1
col: (0,1,2): 0
    0    1    2  
0 ['X', 'X', 'O']
1 ['O', 'O', '-']
2 ['X', '-', '-']


    0    1    2  
0 ['X', 'X', 'O']
1 ['O', 'O', 'X']
2 ['X', '-', '-']


row: (0,1,2): 2
col: (0,1,2): 1
    0    1    2  
0 ['X', 'X', 'O']
1 ['O', 'O', 'X']
2 ['X', 'O', '-']


    0    1    2  
0 ['X', 'X', 'O']
1 ['O', 'O', 'X']
2 ['X', 'O', 'X']


BREAK
